In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/chang/data

!pip install deepface
from deepface import DeepFace
import sqlite3

Mounted at /content/drive
/content/drive/MyDrive/chang/data
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.0 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116952 sha256=a19cfbd9dc4cdc4bc59b7d818ce7f0cf1a6978f5b2dac97d46d85f90b5adcc29
  Stored in directory: /root/.cache/pip/wheels/f7/f1/89/b9ea2bf8f80ec027a88fef1d354b3816b4d3d29530988972f6
Successfully built fire
Directory  /root /.deepface created
Directory  /root /.deepface/weights created


In [2]:
%cd /content/drive/MyDrive/chang/db
conn = sqlite3.connect('people.db')
c = conn.cursor()

/content/drive/MyDrive/chang/db


In [ ]:
c.execute('''
          CREATE TABLE IF NOT EXISTS persons
          (name TEXT, uid INTEGER,
           PRIMARY KEY (name, uid))
          ''')

In [ ]:
from re import I
%cd /content/drive/MyDrive/chang/data

import os
import shutil
import time
import fnmatch

def split_train_test(path, test_path, train_path, name=None):
    output = os.listdir(path)
    global uid, TEST_NUM
    current_test_num = 0
    
    if name:
        person_uid.append((name, uid))

    for i in output:
        cur_path = path+"/"+i
        if os.path.isdir(cur_path):
            split_train_test(cur_path, test_path, train_path, i)
            uid = uid+1
        elif os.path.isfile(cur_path):
            if current_test_num >= TEST_NUM or len(DeepFace.extract_faces(img_path=cur_path, enforce_detection=False))==0:
                shutil.copyfile(cur_path, train_path+"/"+"uid="+str(uid)+"_"+i)
            else:
                shutil.copyfile(cur_path, test_path+"/"+"uid="+str(uid)+"_"+i)  
                current_test_num = current_test_num + 1

    return

/content/drive/MyDrive/chang/data


In [ ]:
def split_data(path, test_path, train_path):
    if not os.path.exists(test_path):
        os.makedirs(test_path) 
    if not os.path.exists(train_path):
        os.makedirs(train_path)

    
    split_train_test(src_path, test_path, train_path)

    conn = sqlite3.connect('persons.db')
    c = conn.cursor()
    c.executemany("INSERT INTO persons VALUES(?, ?)", person_uid)
    conn.commit()
    conn.close()

In [ ]:
src_path = "/content/drive/MyDrive/chang/data" # 기존 폴더 경로
test_path = "/content/drive/MyDrive/chang/test_db" # 옮길 폴더 경로
train_path = "/content/drive/MyDrive/chang/train_db" # 옮길 폴더 경로

uid=0
TEST_NUM = 4
person_uid = []
split_data(src_path, test_path, train_path)